In [3]:
import requests
import csv

# Define the API endpoint and parameters
api_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"
api_key = "<API_Key>"  # Replace with your NREL API key
params = {
    "api_key": api_key,
    "fuel_type": "ELEC",
    "country": "CA",
    "limit": "all"  # Fetch all results
}

try:
    # Make the API request
    response = requests.get(api_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        stations = data.get("fuel_stations", [])

        # Define the CSV file name
        csv_file = "ev_charging_stations_canada.csv"

        # Open the CSV file for writing
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)

            # Write the header
            writer.writerow([
                "Station Name", "Street Address", "City", "State", "Zip",
                "Latitude", "Longitude", "EV Connector Types", "EV Network",
                "Status", "Open Date"
            ])

            # Write station data
            for station in stations:
                ev_connector_types = station.get("ev_connector_types")
                if not isinstance(ev_connector_types, list):  # Ensure it's a list
                    ev_connector_types = []
                writer.writerow([
                    station.get("station_name"),
                    station.get("street_address"),
                    station.get("city"),
                    station.get("state"),
                    station.get("zip"),
                    station.get("latitude"),
                    station.get("longitude"),
                    ", ".join(ev_connector_types),  # Safely join if it's a list
                    station.get("ev_network"),
                    station.get("status_code"),
                    station.get("open_date")
                ])

        print(f"Data has been successfully exported to {csv_file}")
    else:
        print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")


Data has been successfully exported to ev_charging_stations_canada.csv


In [4]:
import pandas as pd
        
ev_charging_stations_canada = pd.read_csv('ev_charging_stations_canada.csv', sep = ',')
ev_charging_stations_canada

,Station Name,Street Address,City,State,Zip,Latitude,Longitude,EV Connector Types,EV Network,Status,Open Date
0,Ramada,1319 2nd St W,Brooks,AB,T1R 1P7,50.585242,-111.898615,J1772,Non-Networked,E,2012-02-01
1,Davis Chevrolet,149 E Lake Crescent NE,Airdrie,AB,T4A 2H9,51.288119,-113.998284,J1772,Non-Networked,E,2015-01-15
2,Gasonic Instruments,8-823 41st Ave NE,Calgary,AB,T2E 6Y3,51.092856,-114.043029,J1772,Non-Networked,E,2015-04-15
3,International Motor Cars,7220 Railway St SE,Calgary,AB,T2H 3A8,50.990495,-114.042414,J1772,Non-Networked,E,2017-03-15
4,Residence Inn,3710 Market St SE,Calgary,AB,T3M 2P2,50.880283,-113.955873,J1772,Non-Networked,E,2017-02-01
...,...,...,...,...,...,...,...,...,...,...,...
14615,Keywest Inn Hotel,71 Superior St,Decon,AB,T9G 1K9,53.361193,-113.731398,J1772,CHARGELAB,E,2024-11-29
14616,Comfort Inn & Suites Goderich,135 Gibbons St,Goderich,ON,N7A 3J5,43.739082,-81.706438,J1772,CHARGELAB,E,2024-11-29
14617,3 Valiquette - Kiamika,25 Rue Principale,Kiamika,QC,J0W 1G0,46.418074,-75.381317,J1772,Circuit électrique,E,2024-11-29
14618,Saint-Étienne-des-Grès - Hôtel de ville,1230 rue Principale,Saint-Étienne-des-Grès,QC,G0X 2P0,46.440472,-72.772612,J1772,Circuit électrique,E,2024-11-29


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# First, let's make sure seaborn is properly configured
sns.set_theme(style="whitegrid")  # This is more reliable than plt.style.use('seaborn')

# Create a directory for the plots if it doesn't exist
output_dir = 'charging_station_plots'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Read the charging station data
df = pd.read_csv('ev_charging_stations_canada.csv')

# Convert Open Date to datetime
df['Open Date'] = pd.to_datetime(df['Open Date'])

# Function to create and save a plot - this helps prevent memory leaks
def create_and_save_plot(plot_func, filename):
    plt.figure(figsize=(12, 8))
    plot_func()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# 1. Geographic Distribution of Charging Stations
def plot_province_distribution():
    province_counts = df['State'].value_counts()
    sns.barplot(x=province_counts.index, y=province_counts.values)
    plt.title('Distribution of Charging Stations by Province', pad=20, fontsize=14)
    plt.xlabel('Province', fontsize=12)
    plt.ylabel('Number of Stations', fontsize=12)
    plt.xticks(rotation=45)

create_and_save_plot(plot_province_distribution, '1_province_distribution.png')

# 2. Charging Connector Types Analysis
def plot_connector_types():
    connector_counts = df['EV Connector Types'].str.split(',').explode().str.strip().value_counts()
    plt.pie(connector_counts.values, labels=connector_counts.index, autopct='%1.1f%%')
    plt.title('Distribution of EV Connector Types', pad=20, fontsize=14)

create_and_save_plot(plot_connector_types, '2_connector_types.png')

# 3. Timeline of Station Openings
def plot_timeline():
    df['Year'] = df['Open Date'].dt.year
    yearly_openings = df['Year'].value_counts().sort_index()
    plt.plot(yearly_openings.index, yearly_openings.values, marker='o')
    plt.title('Timeline of Charging Station Openings', pad=20, fontsize=14)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of New Stations', fontsize=12)
    plt.grid(True)

create_and_save_plot(plot_timeline, '3_timeline.png')

# 4. Network Distribution
def plot_network_distribution():
    network_counts = df['EV Network'].value_counts()
    sns.barplot(x=network_counts.index, y=network_counts.values)
    plt.title('Distribution of Charging Station Networks', pad=20, fontsize=14)
    plt.xlabel('Network Type', fontsize=12)
    plt.ylabel('Number of Stations', fontsize=12)
    plt.xticks(rotation=45)

create_and_save_plot(plot_network_distribution, '4_network_distribution.png')

# 5. Station Density Heatmap
def plot_density_heatmap():
    sns.kdeplot(data=df, x='Longitude', y='Latitude', cmap='YlOrRd', fill=True)
    plt.title('Charging Station Density Heatmap', pad=20, fontsize=14)
    plt.xlabel('Longitude', fontsize=12)
    plt.ylabel('Latitude', fontsize=12)

create_and_save_plot(plot_density_heatmap, '5_density_heatmap.png')

# Generate a detailed summary report
def generate_summary_report():
    with open(os.path.join(output_dir, 'charging_station_analysis_summary.txt'), 'w') as f:
        f.write("EV Charging Infrastructure Analysis Summary\n")
        f.write("=========================================\n\n")

        # Overall statistics
        f.write("1. Overall Statistics:\n")
        f.write(f"Total number of charging stations: {len(df)}\n")
        f.write(f"Number of unique cities: {df['City'].nunique()}\n")
        f.write(f"Date range: {df['Open Date'].min().year} to {df['Open Date'].max().year}\n\n")

        # Provincial distribution
        f.write("2. Provincial Distribution:\n")
        f.write(df['State'].value_counts().to_string())
        f.write("\n\nAverage stations per city: {:.2f}\n\n".format(df.groupby('City').size().mean()))

        # Connector types
        f.write("3. Connector Type Distribution:\n")
        connector_counts = df['EV Connector Types'].str.split(',').explode().str.strip().value_counts()
        f.write(connector_counts.to_string())
        f.write("\n\n")

        # Network analysis
        f.write("4. Network Distribution:\n")
        f.write(df['EV Network'].value_counts().to_string())

generate_summary_report()

print(f"All plots have been saved to the '{output_dir}' directory")
print(f"A summary report has been generated in '{output_dir}/charging_station_analysis_summary.txt'")

All plots have been saved to the 'charging_station_plots' directory
A summary report has been generated in 'charging_station_plots/charging_station_analysis_summary.txt'
